In [2]:
#import libraries
import os
import re
import numpy as np
import pandas as pd
import psycopg2
from unidecode import unidecode

In [3]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is E807-C1C7

 Directory of C:\Users\Administrator\Documents\jupyter_projects

24/07/2023  10:24 pm    <DIR>          .
21/07/2023  11:28 am    <DIR>          ..
24/07/2023  02:03 pm    <DIR>          .ipynb_checkpoints
24/07/2023  02:18 pm    <DIR>          csv_text_cleaner
24/07/2023  10:24 pm            22,734 csv_text_cleaner.ipynb
24/07/2023  02:02 pm            23,356 Untitled.ipynb
24/07/2023  01:40 pm    <DIR>          world_data_2023
24/07/2023  07:10 pm            36,034 world_data_2023.csv
               3 File(s)         82,124 bytes
               5 Dir(s)  169,640,398,848 bytes free


In [4]:
#insert csv file pathway into str
csv_path = str("world_data_2023\world-data-2023.csv")

#split file pathway to split file name into tuple, containing name( pos [0]) and file extension ( pos [1])
pathway = os.path.basename(csv_path)
file_name = os.path.splitext(pathway)

#check to see if file name has been stripped from extension
print(file_name)
print(file_name[0])

df = pd.read_csv(csv_path, thousands =',')
df.head()

('world-data-2023', '.csv')
world-data-2023


,Country,Density\n(P/Km2),Abbreviation,Agricultural Land( %),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,...,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban_population,Latitude,Longitude
0,Afghanistan,60,AF,58.10%,652230.0,323000.0,32.49,93.0,Kabul,8672.0,...,78.40%,0.28,38041754.0,48.90%,9.30%,71.40%,11.12%,9797273.0,33.939110,67.709953
1,Albania,105,AL,43.10%,28748.0,9000.0,11.78,355.0,Tirana,4536.0,...,56.90%,1.20,2854191.0,55.70%,18.60%,36.60%,12.33%,1747593.0,41.153332,20.168331
2,Algeria,18,DZ,17.40%,2381741.0,317000.0,24.28,213.0,Algiers,150006.0,...,28.10%,1.72,43053054.0,41.20%,37.20%,66.10%,11.70%,31510100.0,28.033886,1.659626
3,Andorra,164,AD,40.00%,468.0,NaN,7.20,376.0,Andorra la Vella,469.0,...,36.40%,3.33,77142.0,NaN,NaN,NaN,NaN,67873.0,42.506285,1.521801
4,Angola,26,AO,47.50%,1246700.0,117000.0,40.73,244.0,Luanda,34693.0,...,33.40%,0.21,31825295.0,77.50%,9.20%,49.10%,6.89%,21061025.0,-11.202692,17.873887


In [11]:
def text_cleaner(cleaned_string, units_to_remove=None):
    if units_to_remove is None:
        units_to_remove = []

    replace_dict = {
        '-': '_',
        ' ':'_',
        ':':'',
        '&': '',
        '$':'',
        '%':'',
        '!':'',
        '/':'',
        '(': '',
        ')': '',
        '\\': '',
        '\n': '',
    }

    #remove units from the cleaned name
    for unit in units_to_remove:
        cleaned_string = re.sub(re.escape(unit), '', cleaned_string)

    #remove trailing symbols from the cleaned string
    cleaned_string = re.sub(r'^[_-]+|[_-]+$','',cleaned_string)

    #remove non-latin characters and transliterate with nearest equivalent
    cleaned_string = unidecode(cleaned_string)
    
    cleaned_string = cleaned_string.rstrip().lower().translate(str.maketrans(replace_dict))

    return cleaned_string

df.columns = [text_cleaner(text, units_to_remove=['pkm2','km2','co2']) for text in df.columns]
print(df.columns)

#clean file name
cleaned_title = text_cleaner(file_name[0])
print(cleaned_title)
    


Index(['country', 'density', 'abbreviation', 'agricultural_land', 'land_area',
       'armed_forces_size', 'birth_rate', 'calling_code', 'capitalmajor_city',
       'emissions', 'cpi', 'cpi_change', 'currency_code', 'fertility_rate',
       'forested_area', 'gasoline_price', 'gdp',
       'gross_primary_education_enrollment',
       'gross_tertiary_education_enrollment', 'infant_mortality',
       'largest_city', 'life_expectancy', 'maternal_mortality_ratio',
       'minimum_wage', 'official_language', 'out_of_pocket_health_expenditure',
       'physicians_per_thousand', 'population',
       'population_labor_force_participation', 'tax_revenue', 'total_tax_rate',
       'unemployment_rate', 'urban_population', 'latitude', 'longitude'],
      dtype='object')
world_data_2023


In [8]:
df.dtypes

country                                  object
density                                   int64
abbreviation                             object
agricultural_land                        object
land_area                               float64
armed_forces_size                       float64
birth_rate                              float64
calling_code                            float64
capitalmajor_city                        object
emissions                               float64
cpi                                     float64
cpi_change                               object
currency_code                            object
fertility_rate                          float64
forested_area                            object
gasoline_price                           object
gdp                                      object
gross_primary_education_enrollment       object
gross_tertiary_education_enrollment      object
infant_mortality                        float64
largest_city                            

In [12]:
#replace pandas dataframe datatypes with equivalent SQL datatypes
data_type_replacements = {
    'object':'varchar',
    'float64':'float',
    'int64':'int',
    'datetime64':'timestamp',
    'timedelta64[ns]':'varchar'
}


#zip list of headers and corresponding datatypes so they are in format: '..., {header} {datatype}, ...'
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(data_type_replacements)))
col_str

'country varchar, density int, abbreviation varchar, agricultural_land varchar, land_area float, armed_forces_size float, birth_rate float, calling_code float, capitalmajor_city varchar, emissions float, cpi float, cpi_change varchar, currency_code varchar, fertility_rate float, forested_area varchar, gasoline_price varchar, gdp varchar, gross_primary_education_enrollment varchar, gross_tertiary_education_enrollment varchar, infant_mortality float, largest_city varchar, life_expectancy float, maternal_mortality_ratio float, minimum_wage varchar, official_language varchar, out_of_pocket_health_expenditure varchar, physicians_per_thousand float, population float, population_labor_force_participation varchar, tax_revenue varchar, total_tax_rate varchar, unemployment_rate varchar, urban_population float, latitude float, longitude float'

In [13]:
#PostgreSQL credentials

host = "localhost"
database = "postgres"
user = "postgres"
password = "password"

#establish a connection to the PostgreSQL server

try: 
    
    connection = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password
    )

    print("Connected to PostgreSQL!")
except Exception as e:
    print("Error connecting to PostgreSQL", e)
    exit()

#set client encoding
connection.set_client_encoding('UTF8')


#create a cursor to execute SQL queries

cursor = connection.cursor()

#drop table with same name

try:
    cursor.execute(f"drop table if exists {cleaned_title} CASCADE;")
    print(f"{cleaned_title} table dropped if exists.")
except Exception as e:
    print("Error dropping table", e)
    exit()
    
#create the table in PostgreSQL

try:
    create_table_query = f"CREATE TABLE {cleaned_title} ({col_str});"
    cursor.execute(create_table_query)
    print(f"{cleaned_title} table created!")
except Exception as e:
    print("Error creating table", e)
    exit()

#insert the data into the table
try:
    data_to_insert = [tuple(row) for _, row in df.iterrows()]    
    insert_query = f"INSERT INTO {cleaned_title} ({', '.join(df.columns)}) VALUES ({', '.join(['%s']*len(df.columns))});"
    cursor.executemany(insert_query, data_to_insert)
    print("Data inserted to table")
except Exception as e:
    print("Error inserting data into table", e)
    connection.rollback()
    exit()
else:
    connection.commit()


Connected to PostgreSQL!
world_data_2023 table dropped if exists.
world_data_2023 table created!
Data inserted to table


In [14]:
#save df to csv

csv_output_path = f"{cleaned_title}.csv"
df.to_csv(csv_output_path, header=df.columns, index=False, encoding='utf-8')

#open file in memory
my_file = open(csv_output_path, mode ='r', encoding='utf-8')
print("file opened in memory")

#upload to db
SQL_STATEMENT = f"""
COPY {cleaned_title} FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql=SQL_STATEMENT, file = my_file)
print("file copied to db")

file opened in memory
file copied to db


In [15]:
#change permissions to public
cursor.execute(f"GRANT SELECT ON TABLE {cleaned_title} TO public")
connection.commit()

cursor.close()
print(f"table {cleaned_title} import to database completed")

table world_data_2023 import to database completed
